### Import

In [ ]:
# Libraries
import numpy as np
import casadi as ca
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, FFMpegWriter
from IPython.display import HTML
from mpl_toolkits.mplot3d import Axes3D

# Files
from vehicle import *
from plotting import *
from simulate import *
from mpc import *
from global_path_planning import *
from obstacles import *
 

### Simulation Input Parameters

In [ ]:
#### MPC Properties
dt = 1                  # Time step [s]
N = 10                  # Time Horizon

####  Simulation/Problem Parameters
pos_constraints = [0, 20, 0, 50, 0, 20]         # Position Constraints [m]:    [x_min, x_max, y_min, y_max, z_min, z_max	]
x_init = [10, 2, 2, 0, 0, 0]          # Initial State: [x, y, z, x_dot, y_dot, z_dot]
x_target_last = [10, 48, 2, 0, 0, 0]      # Target State: [x, y, z, x_dot, y_dot, z_dot]
T = 150                                   # Simulation time [s]

#### Drone Velocity & Acceleration Constraints
vel_constraints = [-1, 1, -1, 1, -1, 1]         # Velocity Constraints [m/s]:  [x_min, x_max, y_min, y_max, z_min, z_max] 
acc_constraints = [-10, 10, -10, 10, -10, 10]         # Acceleration Constraints [m/s^2]:    [x_min, x_max, y_min, y_max, z_min, z_max]

#### Obstacles
static_cost = 200
dynamic_cost = 400

waypoint_radius = 5


In [ ]:
#### OBSTACLE DEFINITION 

## Vertical Obstacles
V_obstacles = [[10, 20, 4], [18, 30, 3], [7, 40, 5]]         # [center_x, center_y, radius]
V_move_obstacles = [[20, 40, -0.7, 0, 5], [17, 0, 0, 1, 5]]  # [center_x, center_y, vel_x, vel_y, radius]

H_obstacles = []         # [center_x, center_y, radius]
H_move_obstacles = []  # [center_x, center_y, vel_x, vel_y, radius]

obstacles = []
move_obstacles = []

# Class Creation
for V_obstacle in V_obstacles:
    obstacles.append(CylinderVertical(V_obstacle[0], V_obstacle[1], V_obstacle[2], static_cost))

for H_obstacle in H_obstacles:
    obstacles.append(CylinderHorizontal(H_obstacle[0], H_obstacle[1], H_obstacle[2], static_cost))

for V_move_obstacle in V_move_obstacles:
    move_obstacles.append(CylinderVertical(V_move_obstacle[0], V_move_obstacle[1], V_move_obstacle[4], dynamic_cost, V_move_obstacle[2:4]))

for H_move_obstacle in V_move_obstacles:
    move_obstacles.append(CylinderHorizontal(H_move_obstacle[0], H_move_obstacle[1], H_move_obstacle[4], dynamic_cost, H_move_obstacle[2], H_move_obstacle[3]))


In [ ]:
# Run RRT*
#get global a_star path and visualize
World = World_3D([pos_constraints[0], pos_constraints[1]], [pos_constraints[2], pos_constraints[3]], [pos_constraints[4], pos_constraints[5]], obstacles,  obstacle_margin=0)
World.plot()

start = x_init[:3]
goal = x_target_last[:3]

print("RRT*")
#path_rrt = rrt_star(World, start, goal, radius=10, max_iter=10000)
#path_rrt.plot()


In [ ]:
# Plot the static Obstacles
plot_static_obstacles(obstacles[0], pos_constraints, x_init, x_target_last)

In [ ]:
# Setup vehichle & controller then Simulate
vehicle = vehicle_SS(dt)
controller = lambda x_init, current_wp, move_obstacles : mpc_control(vehicle, N, x_init, current_wp, pos_constraints, vel_constraints, acc_constraints, obstacles, move_obstacles)
states, inputs, plans, timesteps, target_state = simulate(dt, T, x_init, N, controller, move_obstacles, path_rrt, waypoint_radius, num_states=len(x_init), num_inputs=2)

In [ ]:
# def update(frame):
#     # Clear previous frame
#     plt.cla()

#     move_obstacles_update = MovingObstacleConvert(move_obstacles, frame, dt)
    
#     # Plot Path for current state
#     plt.scatter(plans[0][:, frame], plans[1][:, frame], c='g', s=20, label = "Planned Trajectory")

#     # Plot current state
#     drone = plt.Circle((target_state[0][frame], target_state[1][frame]), 0.7, color='b', fill=True, label = "Target Point")
#     plt.gca().add_patch(drone)

#     drone = plt.Circle((states[0][frame], states[1][frame]), 0.7, color='r', fill=True, label = "Drone")
#     plt.gca().add_patch(drone)
    
#     for obstacle in obstacles:
#         circle = plt.Circle((obstacle[0], obstacle[1]), obstacle[2], color='k', fill=False)
#         plt.gca().add_patch(circle) 

#     for obstacle in move_obstacles_update:
#         circle = plt.Circle((obstacle[0], obstacle[1]), obstacle[2], color='k', fill=False)
#         plt.gca().add_patch(circle) 

#     # Env Boundaries
#     x = pos_constraints[0]
#     y = pos_constraints[2]
#     width = pos_constraints[1]-x
#     height = pos_constraints[3]-x
#     rectangle = patches.Rectangle((x, y), width, height, edgecolor='red', facecolor='none')
#     plt.gca().add_patch(rectangle)

#     plt.legend()
#     plt.title(f'Time Step {frame + 1}/{len(states[1])}')  # Set title for each frame
#     plt.xlabel('X-axis')
#     plt.ylabel('Y-axis')
#     plt.xlim(0, 50)
#     plt.ylim(0, 50)
#     plt.grid()

# Create the animation
fig = plt.figure()
ani = FuncAnimation(fig, update, frames=len(states[1]), interval=100)

# Display the animation within the Jupyter Notebook using HTML
HTML(ani.to_jshtml())
